In [2]:
import os
os.chdir('../')

In [3]:
pwd

'c:\\Users\\asbpi\\Desktop\\ASB\\Data_Science\\Projects\\MY Projects\\Stroke-Risk-Prediction'

In [3]:
# Entity

In [4]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_file: Path

In [4]:
# Configuration Manager

In [5]:
from stroke_risk.constants import *
from stroke_risk.utils.common import read_yaml, create_directories

In [6]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_file=config.data_file,
        )

        return data_transformation_config

In [5]:
# Components

In [21]:
import os
from stroke_risk import logger
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import pandas as pd

In [22]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config

    def encoding(self):
        data= pd.read_csv(self.config.data_file)

        label_encoder = LabelEncoder()
        data['gender'] = label_encoder.fit_transform(data['gender']) # male=1, female=0
        data['ever_married'] = label_encoder.fit_transform(data['ever_married']) # yes=1, no=0
        data['Residence_type'] = label_encoder.fit_transform(data['Residence_type']) # urban=1, rural=0

        data = pd.get_dummies(data, columns=['work_type', 'smoking_status'])
        data.to_csv(os.path.join(self.config.root_dir, "data.csv"),index = False)
        
        logger.info('LabelEncoder & OnehotEncoder applied to Categorical Columns')
        logger.info(f'Encoded data saved in {self.config.root_dir}/data.csv')


    def train_test_spliting(self):
        data = pd.read_csv(os.path.join(self.config.root_dir, "data.csv"))

        train, test = train_test_split(data, test_size=0.2)

        train.to_csv(os.path.join(self.config.root_dir, "train.csv"),index = False)
        test.to_csv(os.path.join(self.config.root_dir, "test.csv"),index = False)

        logger.info("Splited data into training and test sets")
        logger.info(f'train data shape - {train.shape}')
        logger.info(f'test data shape - {test.shape}')

In [6]:
# Pipeline

In [23]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.encoding()
    data_transformation.train_test_spliting()
except Exception as e:
    raise e

[2023-12-26 14:19:17,640: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-12-26 14:19:17,640: INFO: common: yaml file: params.yaml loaded successfully]
[2023-12-26 14:19:17,648: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-12-26 14:19:17,648: INFO: common: created directory at: artifacts]
[2023-12-26 14:19:17,648: INFO: common: created directory at: artifacts/data_transformation]
[2023-12-26 14:19:17,685: INFO: 2359033959: LabelEncoder & OnehotEncoder applied to Categorical Columns]
[2023-12-26 14:19:17,694: INFO: 2359033959: Encoded data saved in artifacts/data_transformation/data.csv]
[2023-12-26 14:19:17,787: INFO: 2359033959: Splited data into training and test sets]
[2023-12-26 14:19:17,787: INFO: 2359033959: train data shape - (4052, 16)]
[2023-12-26 14:19:17,787: INFO: 2359033959: test data shape - (1014, 16)]
